# Error Analysis - GeoID-based Models

This notebook performs error analysis for GeoID-based prediction models to identify when and where models fail.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import sys
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from contextlib import redirect_stdout
from io import StringIO

warnings.filterwarnings('ignore')
os.makedirs('GeoID_error_analysis', exist_ok=True)

# Set up output capture to save all prints to file
output_file = open('GeoID_error_analysis/GeoID_error_analysis.txt', 'w')
output_capture = StringIO()

# Custom print function that prints to both console and file
class TeeOutput:
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush()
    def flush(self):
        for f in self.files:
            f.flush()

# Redirect stdout to both console and file
original_stdout = sys.stdout
sys.stdout = TeeOutput(sys.stdout, output_file)

print("=" * 70)
print("GeoID ERROR ANALYSIS - RF Models")
print("=" * 70)
print()


GeoID ERROR ANALYSIS - RF Models



Test dataframes created:
   Daily: 71568 samples
   Weekly: 20017 samples
   Monthly: 4409 samples
1. GeoID-wise MAE Analysis

Daily - Top 5 GeoIDs by MAE:
Geo ID
4.845398e+11    3.116278
4.845304e+11    0.541581
4.845305e+11    0.396464
4.845304e+11    0.318532
4.849102e+11    0.264210
Name: Abs_Error_RF, dtype: float64


Weekly - Top 5 GeoIDs by MAE:
Geo ID
4.845398e+11    24.342606
4.845304e+11     5.096468
4.845305e+11     4.507922
4.845304e+11     3.391483
4.845304e+11     3.305798
Name: Abs_Error_RF, dtype: float64


Monthly - Top 5 GeoIDs by MAE:
Geo ID
4.845398e+11    59.182730
4.845304e+11    13.893542
4.845305e+11    12.053464
4.845304e+11    11.500145
4.849102e+11     9.458793
Name: Abs_Error_RF, dtype: float64

2. Peak vs Normal Demand Analysis
Daily - Normal: 0.154, Peak: 0.148
Weekly - Normal: 0.693, Peak: 3.708
Monthly - Normal: 2.729, Peak: 9.336

3. Worst-5 Failure Cases (Daily)
      Geo ID       Date  Actual   Pred_RF  Abs_Error_RF  pct_priority_1  pct_mental_health


## Load Models and Data

Load trained models and test data for error analysis.


In [2]:
# Load enhanced datasets
daily_data = pd.read_csv('geoid_daily_enhanced.csv')
weekly_data = pd.read_csv('geoid_weekly_enhanced.csv')
monthly_data = pd.read_csv('geoid_monthly_enhanced.csv')

# Load RF models only
rf_daily = joblib.load('Geoid_Model_Comparison_Results/models/rf_enhanced_daily.joblib')
rf_weekly = joblib.load('Geoid_Model_Comparison_Results/models/rf_enhanced_weekly.joblib')
rf_monthly = joblib.load('Geoid_Model_Comparison_Results/models/rf_enhanced_monthly.joblib')


## Prepare Test Data

Split data and prepare features for predictions.


In [3]:
# Encode GeoIDs (must use same encoder as training)
le_geoid = LabelEncoder()
daily_data['GeoID_Encoded'] = le_geoid.fit_transform(daily_data['Geo ID'])
weekly_data['GeoID_Encoded'] = le_geoid.transform(weekly_data['Geo ID'])
monthly_data['GeoID_Encoded'] = le_geoid.transform(monthly_data['Geo ID'])

# Daily features (must match training exactly)
enhanced_features_daily = ['Month', 'Year', 'Day_of_Year', 'Week', 
                           'pct_priority_1', 'pct_priority_2', 'pct_priority_3', 'pct_priority_4',
                           'pct_mental_health',
                           'pct_category_1', 'pct_category_2', 'pct_category_3', 'pct_category_4', 'pct_category_5',
                           'lag_previous_day', 'lag_same_day_last_week', 'lag_2days_ago', 
                           'lag_same_day_last_month', 'lag_previous_week_total',
                           'is_weekend', 'is_peak_day', 'is_holiday',
                           'is_peak_hour_period', 'hour_category', 'hours_from_peak',
                           'pct_priority_1_peak_hour', 'is_high_priority_day', 'priority_1_x_peak_hour',
                           'days_since_start', 'year_trend', 'month_trend', 'week_trend', 'day_of_year_trend',
                           'rolling_mean_7d', 'rolling_std_7d', 'rolling_mean_30d',
                           'GeoID_Encoded']

# Weekly features
enhanced_features_weekly = ['Month', 'Year', 'Week',
                           'pct_priority_1', 'pct_priority_2', 'pct_priority_3', 'pct_priority_4',
                           'pct_mental_health',
                           'pct_category_1', 'pct_category_2', 'pct_category_3', 'pct_category_4', 'pct_category_5',
                           'lag_previous_week',
                           'is_peak_hour_period', 'hours_from_peak',
                           'is_high_priority_week',
                           'days_since_start', 'year_trend', 'month_trend', 'week_trend',
                           'rolling_mean_4w',
                           'GeoID_Encoded']

# Monthly features
enhanced_features_monthly = ['Month', 'Year',
                            'pct_priority_1', 'pct_priority_2', 'pct_priority_3', 'pct_priority_4',
                            'pct_mental_health',
                            'pct_category_1', 'pct_category_2', 'pct_category_3', 'pct_category_4', 'pct_category_5',
                            'lag_previous_month', 'lag_same_month_last_year',
                            'is_peak_hour_period', 'hours_from_peak',
                            'is_high_priority_month', 'is_peak_month',
                            'days_since_start', 'year_trend', 'month_trend',
                            'rolling_mean_3m',
                            'GeoID_Encoded']

# Split data (same split as training: temporal, 80/20, random_state=42)
X_daily = daily_data[enhanced_features_daily]
y_daily = daily_data['Call_Count']
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_daily, y_daily, test_size=0.2, shuffle=False, random_state=42)

X_weekly = weekly_data[enhanced_features_weekly]
y_weekly = weekly_data['Call_Count']
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(
    X_weekly, y_weekly, test_size=0.2, shuffle=False, random_state=42)

X_monthly = monthly_data[enhanced_features_monthly]
y_monthly = monthly_data['Call_Count']
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_monthly, y_monthly, test_size=0.2, shuffle=False, random_state=42)

# Get RF predictions only
pred_rf_d = np.maximum(0, rf_daily.predict(X_test_d))
pred_rf_w = np.maximum(0, rf_weekly.predict(X_test_w))
pred_rf_m = np.maximum(0, rf_monthly.predict(X_test_m))


## Error Analysis - 3 Key Analyses

1. GeoID-wise MAE (Daily, Weekly, Monthly)
2. Peak vs Normal Demand (Daily, Weekly, Monthly)  
3. Worst-5 Failure Cases (Daily only)


In [4]:
# Create test results dataframes for all aggregations
# Daily
test_daily_with_geoid = X_test_d.copy()
test_daily_with_geoid['Geo ID'] = le_geoid.inverse_transform(X_test_d['GeoID_Encoded'])
test_daily_with_geoid['Actual'] = y_test_d.values
test_daily_with_geoid['Pred_RF'] = pred_rf_d
test_daily_with_geoid['Abs_Error_RF'] = np.abs(test_daily_with_geoid['Actual'] - test_daily_with_geoid['Pred_RF'])

# Add Date and other features needed for analysis
test_indices = X_test_d.index
test_daily_with_geoid['Date'] = daily_data.loc[test_indices, 'Date'].values
test_daily_with_geoid['is_peak_day'] = X_test_d['is_peak_day'].values
test_daily_with_geoid['pct_priority_1'] = X_test_d['pct_priority_1'].values
test_daily_with_geoid['pct_mental_health'] = X_test_d['pct_mental_health'].values

# Weekly
test_weekly_with_geoid = X_test_w.copy()
test_weekly_with_geoid['Geo ID'] = le_geoid.inverse_transform(X_test_w['GeoID_Encoded'])
test_weekly_with_geoid['Actual'] = y_test_w.values
test_weekly_with_geoid['Pred_RF'] = pred_rf_w
test_weekly_with_geoid['Abs_Error_RF'] = np.abs(test_weekly_with_geoid['Actual'] - test_weekly_with_geoid['Pred_RF'])

# Monthly
test_monthly_with_geoid = X_test_m.copy()
test_monthly_with_geoid['Geo ID'] = le_geoid.inverse_transform(X_test_m['GeoID_Encoded'])
test_monthly_with_geoid['Actual'] = y_test_m.values
test_monthly_with_geoid['Pred_RF'] = pred_rf_m
test_monthly_with_geoid['Abs_Error_RF'] = np.abs(test_monthly_with_geoid['Actual'] - test_monthly_with_geoid['Pred_RF'])

# For weekly and monthly, create peak indicator (top 20% of actual values)
test_weekly_with_geoid['is_peak_day'] = (test_weekly_with_geoid['Actual'] >= test_weekly_with_geoid['Actual'].quantile(0.8)).astype(int)
test_monthly_with_geoid['is_peak_day'] = (test_monthly_with_geoid['Actual'] >= test_monthly_with_geoid['Actual'].quantile(0.8)).astype(int)

print(f"Test dataframes created:")
print(f"   Daily: {len(test_daily_with_geoid)} samples")
print(f"   Weekly: {len(test_weekly_with_geoid)} samples")
print(f"   Monthly: {len(test_monthly_with_geoid)} samples")


In [7]:
# ================================
# 1. GEOID-WISE MAE (ALL 3 AGGREGATIONS)
# ================================
def geoid_mae(df, label):
    """Calculate and visualize GeoID-wise MAE for a given aggregation."""
    out = df.groupby('Geo ID')['Abs_Error_RF'].mean().sort_values(ascending=False)
    # Save top 20 GeoIDs for visualization (too many to show all)
    out_top20 = out.head(20)
    out.to_csv(f'GeoID_error_analysis/{label}_geoid_mae.csv')
    
    plt.figure(figsize=(12, 6))
    out_top20.plot(kind='bar')
    plt.ylabel('MAE (RF)')
    plt.title(f'{label.upper()} GeoID-wise MAE (Top 20)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f'GeoID_error_analysis/{label}_geoid_mae.png', dpi=300)
    plt.close()
    return out

print("1. GeoID-wise MAE Analysis")
print("=" * 50)
geoid_mae_daily = geoid_mae(test_daily_with_geoid, 'daily')
print(f"\nDaily - Top 5 GeoIDs by MAE:")
print(geoid_mae_daily.head(5))
print()

geoid_mae_weekly = geoid_mae(test_weekly_with_geoid, 'weekly')
print(f"\nWeekly - Top 5 GeoIDs by MAE:")
print(geoid_mae_weekly.head(5))
print()

geoid_mae_monthly = geoid_mae(test_monthly_with_geoid, 'monthly')
print(f"\nMonthly - Top 5 GeoIDs by MAE:")
print(geoid_mae_monthly.head(5))

# ================================
# 2. PEAK vs NORMAL DEMAND (ALL 3 AGGREGATIONS)
# ================================
def peak_vs_normal(df, label):
    """Calculate and visualize peak vs normal demand errors."""
    out = df.groupby('is_peak_day')['Abs_Error_RF'].mean()
    out.to_csv(f'GeoID_error_analysis/{label}_peak_vs_normal.csv')
    
    plt.figure(figsize=(5, 4))
    out.plot(kind='bar')
    plt.xticks([0, 1], ['Normal', 'Peak'], rotation=0)
    plt.ylabel('MAE (RF)')
    plt.title(f'{label.upper()} GeoID Peak vs Normal Error')
    plt.tight_layout()
    plt.savefig(f'GeoID_error_analysis/{label}_peak_vs_normal.png', dpi=300)
    plt.close()
    return out

print("\n2. Peak vs Normal Demand Analysis")
print("=" * 50)
peak_daily = peak_vs_normal(test_daily_with_geoid, 'daily')
print(f"Daily - Normal: {peak_daily[0]:.3f}, Peak: {peak_daily[1]:.3f}")

peak_weekly = peak_vs_normal(test_weekly_with_geoid, 'weekly')
print(f"Weekly - Normal: {peak_weekly[0]:.3f}, Peak: {peak_weekly[1]:.3f}")

peak_monthly = peak_vs_normal(test_monthly_with_geoid, 'monthly')
print(f"Monthly - Normal: {peak_monthly[0]:.3f}, Peak: {peak_monthly[1]:.3f}")

# ================================
# 3. WORST-5 FAILURE CASES (DAILY ONLY)
# ================================
worst_5 = test_daily_with_geoid.nlargest(5, 'Abs_Error_RF')[
    ['Geo ID', 'Date', 'Actual', 'Pred_RF', 'Abs_Error_RF',
     'pct_priority_1', 'pct_mental_health']
]

worst_5.to_csv('GeoID_error_analysis/daily_worst_5_cases.csv', index=False)

print("\n3. Worst-5 Failure Cases (Daily)")
print("=" * 50)
print(worst_5.to_string(index=False))

# ================================
# SAVE FULL PREDICTION TABLES
# ================================
test_daily_with_geoid.to_csv('GeoID_error_analysis/daily_predictions_with_error.csv', index=False)
test_weekly_with_geoid.to_csv('GeoID_error_analysis/weekly_predictions_with_error.csv', index=False)
test_monthly_with_geoid.to_csv('GeoID_error_analysis/monthly_predictions_with_error.csv', index=False)

print("\nGeoID ERROR ANALYSIS COMPLETE")
print("Generated files:")
files = [f for f in os.listdir('GeoID_error_analysis') if f.endswith('.csv') or f.endswith('.png') or f.endswith('.txt')]
for f in sorted(files):
    print(f"   • {f}")

# Close output file and restore stdout
sys.stdout = original_stdout
output_file.close()
print("\nAll output saved to: GeoID_error_analysis/GeoID_error_analysis.txt")


1. GeoID-wise MAE Analysis

Daily - Top 5 GeoIDs by MAE:
Geo ID
4.845398e+11    3.116278
4.845304e+11    0.541581
4.845305e+11    0.396464
4.845304e+11    0.318532
4.849102e+11    0.264210
Name: Abs_Error_RF, dtype: float64


Weekly - Top 5 GeoIDs by MAE:
Geo ID
4.845398e+11    24.342606
4.845304e+11     5.096468
4.845305e+11     4.507922
4.845304e+11     3.391483
4.845304e+11     3.305798
Name: Abs_Error_RF, dtype: float64


Monthly - Top 5 GeoIDs by MAE:
Geo ID
4.845398e+11    59.182730
4.845304e+11    13.893542
4.845305e+11    12.053464
4.845304e+11    11.500145
4.849102e+11     9.458793
Name: Abs_Error_RF, dtype: float64

2. Peak vs Normal Demand Analysis
Daily - Normal: 0.154, Peak: 0.148
Weekly - Normal: 0.693, Peak: 3.708
Monthly - Normal: 2.729, Peak: 9.336

3. Worst-5 Failure Cases (Daily)
      Geo ID       Date  Actual   Pred_RF  Abs_Error_RF  pct_priority_1  pct_mental_health
4.845398e+11 2023-03-09      31 15.828283     15.171717        0.000000           0.000000
4.845398

In [6]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("GeoID_error_analysis/daily_worst_5_cases.csv")
df["Label"] = df["Geo ID"].astype(str) + " | " + df["Date"]

plt.figure(figsize=(9,5))

ACTUAL_COLOR = "black"
PRED_COLOR = "orange"
LINE_COLOR = "gray"

for i in range(len(df)):
    # Predicted = circle (same color for all)
    plt.scatter(df["Pred_RF"].iloc[i], i, marker="o",
                s=90, color=PRED_COLOR)

    # Actual = cross (same color for all)
    plt.scatter(df["Actual"].iloc[i], i, marker="x",
                s=90, color=ACTUAL_COLOR)

    # Connecting line
    plt.plot(
        [df["Pred_RF"].iloc[i], df["Actual"].iloc[i]],
        [i, i],
        color=LINE_COLOR,
        linewidth=1.5
    )

plt.yticks(range(len(df)), df["Label"])
plt.xlabel("Call Count")
plt.title("Worst 5 RF Failures (Daily GeoID): Actual vs Predicted")

# Clean legend
plt.scatter([], [], marker="x", color=ACTUAL_COLOR, label="Actual")
plt.scatter([], [], marker="o", color=PRED_COLOR, label="Predicted")
plt.legend()

plt.grid(axis="x", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.savefig("GeoID_error_analysis/worst_5_dumbbell_final.png", dpi=300)
plt.close()
